<a href="https://colab.research.google.com/github/FernandoSimon22/vision_artificial/blob/main/YOLOv5_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Clone GitHub repository, install dependencies and check PyTorch and GPU.

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install
%pip install -q roboflow

import torch
import utils
import os
from IPython.display import Image, clear_output
display = utils.notebook_init()  # checks


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="lX9sE2RPQOCv2R7cn0io")
project = rf.workspace("jota22").project("am_boundingbox")
version = project.version(14)
dataset = version.download("yolov5")

In [ ]:
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

### Train YOLOv5s

In [ ]:
# Train YOLOv5s from scracht for 300 epochs
!python train.py --img 640 --batch 16 --epochs 300 --data /content/yolov5/yolov5/AM_BoundingBox-14/data.yaml --weights yolov5s.pt --cache

# División en 5 folds

In [ ]:
import os
import shutil
import random
from sklearn.model_selection import KFold
from pathlib import Path

# Configura tu ruta de dataset aquí
DATASET_PATH = "/content/yolov5/yolov5/AM_BoundingBox-14/train"
IMAGES_DIR = os.path.join(DATASET_PATH, "images")
LABELS_DIR = os.path.join(DATASET_PATH, "labels")

# Número de folds
K_FOLDS = 5

# Lista de todas las imágenes y sus etiquetas
images = sorted([f for f in os.listdir(IMAGES_DIR) if f.endswith(".jpg") or f.endswith(".png")])
labels = sorted([f for f in os.listdir(LABELS_DIR) if f.endswith(".txt")])

# Comprobación
assert len(images) == len(labels), "Número de imágenes y etiquetas no coincide"

# Crear los folds
kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(images)):
    print(f"Preparando Fold {fold + 1}")

    fold_dir = f"/content/folds/fold{fold + 1}"
    train_img_dir = os.path.join(fold_dir, "train/images")
    train_lbl_dir = os.path.join(fold_dir, "train/labels")
    val_img_dir = os.path.join(fold_dir, "val/images")
    val_lbl_dir = os.path.join(fold_dir, "val/labels")

    os.makedirs(train_img_dir, exist_ok=True)
    os.makedirs(train_lbl_dir, exist_ok=True)
    os.makedirs(val_img_dir, exist_ok=True)
    os.makedirs(val_lbl_dir, exist_ok=True)

    for i in train_idx:
        shutil.copy(os.path.join(IMAGES_DIR, images[i]), os.path.join(train_img_dir, images[i]))
        shutil.copy(os.path.join(LABELS_DIR, labels[i]), os.path.join(train_lbl_dir, labels[i]))

    for i in val_idx:
        shutil.copy(os.path.join(IMAGES_DIR, images[i]), os.path.join(val_img_dir, images[i]))
        shutil.copy(os.path.join(LABELS_DIR, labels[i]), os.path.join(val_lbl_dir, labels[i]))

print("✅ División en 5 folds completada")

Preparando Fold 1
Preparando Fold 2
Preparando Fold 3
Preparando Fold 4
Preparando Fold 5
✅ División en 5 folds completada


# Validación cruzada (5-fold CV) con YOLOv5 y calcular las métricas promedio.

# ✅ 1. Configuración inicial

In [ ]:
import os
import shutil
from sklearn.model_selection import KFold
import glob
from pathlib import Path
import yaml

# Rutas
IMAGES_DIR = '/content/yolov5/yolov5/AM_BoundingBox-14/train/images'
LABELS_DIR = '/content/yolov5/yolov5/AM_BoundingBox-14/train/labels'
FOLDS_DIR = '/content/folds'
os.makedirs(FOLDS_DIR, exist_ok=True)

# Archivos
image_files = sorted(glob.glob(os.path.join(IMAGES_DIR, '*.jpg')))
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# ✅ 2. Crear carpetas para cada fold

In [ ]:
for fold, (train_idx, val_idx) in enumerate(kf.split(image_files), 1):
    fold_path = os.path.join(FOLDS_DIR, f'fold{fold}')
    train_img = os.path.join(fold_path, 'images', 'train')
    val_img = os.path.join(fold_path, 'images', 'val')
    train_lbl = os.path.join(fold_path, 'labels', 'train')
    val_lbl = os.path.join(fold_path, 'labels', 'val')
    for d in [train_img, val_img, train_lbl, val_lbl]: os.makedirs(d, exist_ok=True)

    for i in train_idx:
        img = image_files[i]
        lbl = os.path.join(LABELS_DIR, Path(img).stem + '.txt')
        shutil.copy(img, train_img)
        shutil.copy(lbl, train_lbl)

    for i in val_idx:
        img = image_files[i]
        lbl = os.path.join(LABELS_DIR, Path(img).stem + '.txt')
        shutil.copy(img, val_img)
        shutil.copy(lbl, val_lbl)

# ✅ 3. Ejecutar entrenamiento en cada fold

In [ ]:
import subprocess

results = []

for fold in range(1, 6):
    fold_path = f'{FOLDS_DIR}/fold{fold}'

    # Crear YAML
    data_yaml = {
        'train': os.path.join(fold_path, 'images', 'train'),
        'val': os.path.join(fold_path, 'images', 'val'),
        'nc': 1,
        'names': ['def']
    }
    yaml_path = os.path.join(fold_path, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml, f)

    print(f'🔁 Entrenando fold {fold}...')

    result = subprocess.run([
        'python', 'train.py',
        '--img', '640',
        '--batch', '16',
        '--epochs', '300',
        '--data', yaml_path,
        '--weights', 'yolov5s.pt',
        '--project', '/content/folds/results',
        '--name', f'fold{fold}',
        '--exist-ok'
    ])
    results.append(result)

# ✅ 4. Promediar métricas de rendimiento (mAP, precisión, recall)

In [ ]:
import json

maps = []
precisions = []
recalls = []

for fold in range(1, 6):
    path = f'/content/folds/results/fold{fold}/results.txt'
    with open(path, 'r') as f:
        lines = f.readlines()
        last = lines[-1].strip().split()
        maps.append(float(last[2]))       # mAP_0.5
        precisions.append(float(last[0])) # precision
        recalls.append(float(last[1]))    # recall

print(f'📊 Promedio mAP@0.5: {sum(maps)/5:.4f}')
print(f'📊 Promedio precisión: {sum(precisions)/5:.4f}')
print(f'📊 Promedio recall: {sum(recalls)/5:.4f}')

In [ ]:
# YOLOv5 PyTorch HUB Inference (DetectionModels only)
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True, trust_repo=True)  # or yolov5n - yolov5x6 or custom
im = 'https://ultralytics.com/images/zidane.jpg'  # file, Path, PIL.Image, OpenCV, nparray, list
results = model(im)  # inference
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.